In [76]:
import pandas as pd 
import geopandas as gpd


# kemical analysis of water 

In [10]:
kemi = pd.read_csv("/Users/alteafogh/Documents/ITU/GDS/exam/Bathing_Water_2.csv")

filtering for bathing station 

we can decide if we want to have a filter for a specific place (f.ex, just kbh comune, or something like that)
there are a few more we could drop, if we decide for it, such as the enhed or whatever, depending on what we want to keep int the data to work with. mostly as it is irrelevant for what we need at the end (aka, date, and blue flag, with name of the place)

In [11]:
kemi.columns

Index(['﻿Stedtype', 'StedID', 'Stedtekst', 'Referencer', 'GeoZone',
       'x-koordinat', 'y-koordinat', 'Kommune', 'Region', 'Type', 'Medie',
       'Vandområde', 'Dataejer', 'Ansvarlig Myndighed', 'Link', 'Dato',
       'Måling nr', 'Undersøgelsestype', 'Målested type', 'Målested navn',
       'Målested GeoZone', 'Målested, x-koordinat', 'Målested, y-koordinat',
       'Prøvetype', 'Status', 'Analysefraktion', 'CASnr', 'ScKode',
       'Stofparameter', 'Resultat-attribut', 'Resultat', 'Enhed',
       'Kvantifikationsgrænse', 'Detektionsgrænse', 'Kvalitetsmærke'],
      dtype='object')

In [12]:
kemi.dtypes

Stedtype                 object
StedID                    object
Stedtekst                 object
Referencer                object
GeoZone                    int64
x-koordinat              float64
y-koordinat              float64
Kommune                   object
Region                    object
Type                      object
Medie                     object
Vandområde                object
Dataejer                  object
Ansvarlig Myndighed       object
Link                      object
Dato                      object
Måling nr                  int64
Undersøgelsestype        float64
Målested type             object
Målested navn             object
Målested GeoZone           int64
Målested, x-koordinat    float64
Målested, y-koordinat    float64
Prøvetype                 object
Status                    object
Analysefraktion           object
CASnr                    float64
ScKode                     int64
Stofparameter             object
Resultat-attribut         object
Resultat   

In [13]:
kemi = kemi[kemi["Målested type"]=="Badevandsstation"]


dropping columns I believe are unnecessary, so we have a smaller dataset. a lot of the drops were either duplicates, such as coordinates and names, or things I didn't think were important to keep

In [14]:
kemi = kemi.drop(kemi.columns[0], axis=1)
kemi= kemi.drop(columns=[ "Referencer", "Region","Dataejer","Ansvarlig Myndighed", "Link","Undersøgelsestype","CASnr", "ScKode","Kvantifikationsgrænse", "GeoZone",'x-koordinat','y-koordinat','Måling nr', "Medie", "Målested navn" ], axis=1)

In [15]:
kemi["Dato"]= pd.to_datetime(kemi["Dato"], format="%d-%m-%Y")
kemi = kemi.sort_values(by=["Dato"], ascending=False)

In [16]:
kemi["Good_water"] = kemi["Resultat"] <= kemi["Detektionsgrænse"]


In [18]:
kemi.Kommune.unique()

array(['Dragør', 'København', 'Tårnby', 'Gentofte', 'Hvidovre'],
      dtype=object)

In [19]:
kemi.to_csv("Bathing_water_cleaned2.csv")

# now for the puls data

In [51]:
bath = pd.read_csv("/Users/alteafogh/Documents/ITU/GDS/exam/puls-vBadevandsanlaegStamdata.csv", sep=";")

In [52]:
bath.columns

Index(['BathingwaterStationId', 'Name', 'Latitude', 'Longitude', 'Location',
       'Responsible', 'Type', 'WaterType', 'WaterArea', 'WaterAreaNumber',
       'Started', 'Closed', 'BlueFlag', 'ProfileUrl', 'Main'],
      dtype='object')

In [53]:
kommuner_to_keep = ['Dragør', 'København', 'Tårnby', 'Gentofte', 'Hvidovre']
kommuner_to_keep = [k.lower() for k in kommuner_to_keep]
bath['Responsible_lower'] = bath['Responsible'].str.lower()
bath_filtered = bath[bath['Responsible_lower'].apply(lambda x: any(k in x for k in kommuner_to_keep))]
bath_filtered = bath_filtered.drop(columns='Responsible_lower')

In [54]:
bath_filtered = bath_filtered[bath_filtered["Type"]=="Badevandsstation"]

In [55]:
bath_filtered

,BathingwaterStationId,Name,Latitude,Longitude,Location,Responsible,Type,WaterType,WaterArea,WaterAreaNumber,Started,Closed,BlueFlag,ProfileUrl,Main
9,be6b1aa1-c2bc-4da6-ab62-7f1937360850,Ved Kongelundskroen,"55,56278059475752","12,56134220358073",NaN,DRAGØR KOMMUNE,Badevandsstation,Marin,Køge Bugt,DK_Coast7,31.12.1990 00.00.00,NaN,False,https://b0301proddatasto.blob.core.windows.net...,NaN
10,414eced6-2a07-4e81-9aed-c6fa81f9f9a0,Amager Sydstrand,"55,58107234553681","12,658562241276446",NaN,DRAGØR KOMMUNE,Badevandsstation,Marin,Køge Bugt,DK_Coast7,31.12.1990 00.00.00,NaN,False,https://b0301proddatasto.blob.core.windows.net...,NaN
11,6b0da2ae-4e80-477c-8d00-bbe2ec7c385c,Dragør Nordstrand,"55,6034555531332","12,673227748026212",NaN,DRAGØR KOMMUNE,Badevandsstation,Marin,Nordlige Øresund,DK_Coast6,31.12.1990 00.00.00,NaN,False,https://b0301proddatasto.blob.core.windows.net...,NaN
12,51179f3a-1808-4779-8f87-bd170dd6507f,"Dragør Søbad, Syd","55,58665573485622","12,675403549796474",NaN,DRAGØR KOMMUNE,Badevandsstation,Marin,Mangler,Mangler,31.12.1990 00.00.00,31.12.1999 00.00.00,False,NaN,NaN
13,fddb4975-a539-4634-b90d-95d672762bd4,Søvang,"55,56501192136597","12,621983729910687",NaN,DRAGØR KOMMUNE,Badevandsstation,Marin,Køge Bugt,DK_Coast7,31.12.1990 00.00.00,NaN,False,https://b0301proddatasto.blob.core.windows.net...,NaN
14,7dadbe61-dd2e-4433-84e6-9c714beeaef2,"Dragør søbad, nord","55,58702896235373","12,675851272827456",NaN,DRAGØR KOMMUNE,Badevandsstation,Marin,Køge Bugt,DK_Coast7,31.12.1990 00.00.00,NaN,False,https://b0301proddatasto.blob.core.windows.net...,NaN
22,80e232b1-a2ec-4754-83bd-2d080d1ce79b,Hellerup Strand,"55,73263178861659","12,582466140588627",NaN,GENTOFTE KOMMUNE,Badevandsstation,Marin,Nordlige Øresund,DK_Coast6,01.01.1990 00.00.00,NaN,False,https://b0301proddatasto.blob.core.windows.net...,NaN
23,0a112b81-09e5-431e-803b-319ece44a72c,Skovshoved Syd,"55,757942420660335","12,599509939936606",Ved badebroen,GENTOFTE KOMMUNE,Badevandsstation,Marin,Nordlige Øresund,DK_Coast6,01.01.2020 00.00.00,NaN,False,https://b0301proddatasto.blob.core.windows.net...,NaN
25,fa8c2c2b-73e4-4be1-91ae-203636a0443d,Charlottenlund Strandpark,"55,74361092450652","12,58800020904251",NaN,GENTOFTE KOMMUNE,Badevandsstation,Marin,Nordlige Øresund,DK_Coast6,31.12.1990 00.00.00,NaN,False,https://b0301proddatasto.blob.core.windows.net...,NaN
26,77689a39-7915-48bb-ad41-dc7c632b8362,Bellevue strand,"55,77668612950274","12,592493982366353",NaN,GENTOFTE KOMMUNE,Badevandsstation,Marin,Nordlige Øresund,DK_Coast6,31.12.1990 00.00.00,NaN,False,https://b0301proddatasto.blob.core.windows.net...,NaN


In [56]:
bath_filtered.WaterType.unique()

array(['Marin'], dtype=object)

In [57]:
bath2 = bath_filtered.drop(columns=["ProfileUrl", "Main", "BathingwaterStationId","WaterType", "WaterAreaNumber" ])

In [61]:
bath2 = bath2[bath2["Closed"].isna()]

In [ ]:
bath2.to_csv("bathing_places_cleaned.csv")

# joining two datasets

Badevandstamdata has coordinate system: EPSG:25832
kemidata has the coordinate system EPSG:25832

so I am going to join the two datasets together:


In [77]:
df_kemi = pd.read_csv("Bathing_water_cleaned2.csv")
df_bath = pd.read_csv("bathing_places.csv")

In [85]:
df_bath["Latitude"] = df_bath["Latitude"].str.replace(",",".")
df_bath["Longitude"] = df_bath["Longitude"].str.replace(",",".")

In [88]:
df_bath["Latitude"] = pd.to_numeric(df_bath["Latitude"])
df_bath["Longitude"] = pd.to_numeric(df_bath["Longitude"])

In [80]:
gdf_kemi = gpd.GeoDataFrame(df_kemi, geometry=gpd.points_from_xy(df_kemi["Målested, x-koordinat"], df_kemi["Målested, y-koordinat"]), crs="EPSG:25832")

In [93]:
gdf_bath = gpd.GeoDataFrame(df_bath, geometry=gpd.points_from_xy(df_bath["Longitude"],df_bath["Latitude"]), crs="EPSG:25832")

# don't think we should do this, but just in case

In [96]:
gdf_joined = gpd.sjoin(gdf_kemi, gdf_bath, how="left")

In [97]:
gdf_joined

,Unnamed: 0_left,StedID,Stedtekst,Kommune,Type_left,Vandområde,Dato,Målested type,Målested GeoZone,"Målested, x-koordinat",...,Name,Latitude,Longitude,Location,Responsible,Type_right,WaterArea,Started,Closed,BlueFlag
0,15458,7DADBE61-DD2E-4433-84E6-9C714BEEAEF2,"Dragør søbad, nord",Dragør,Badevandsstation,Køge Bugt,2025-04-16,Badevandsstation,32,731635.999999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15634,7DADBE61-DD2E-4433-84E6-9C714BEEAEF2,"Dragør søbad, nord",Dragør,Badevandsstation,Køge Bugt,2025-04-16,Badevandsstation,32,731635.999999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13659,DC6E765D-44AF-4F4F-AA2E-90432C57410A,Islands Brygge Havnebad,København,Badevandsstation,København Havn,2025-04-10,Badevandsstation,32,724995.999999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,779,CE05AC8C-288C-4D22-9EDC-2633DCFB95B3,Byskoven Badezone,København,Badevandsstation,NaN,2025-04-10,Badevandsstation,32,721826.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16515,4FFFFE86-4C75-4D81-B639-9E7700E24E47,Sandkaj Badezone,København,Badevandsstation,København Havn,2025-04-10,Badevandsstation,32,726027.999999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18508,21008,8F1EAC66-DB32-4BC3-AAC5-F9A522442897,Kastrup søbad,Tårnby,Badevandsstation,Mangler,1990-05-15,Badevandsstation,32,730169.999999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18509,15102,7DADBE61-DD2E-4433-84E6-9C714BEEAEF2,"Dragør søbad, nord",Dragør,Badevandsstation,Køge Bugt,1990-05-07,Badevandsstation,32,731635.999999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18510,20968,8F1EAC66-DB32-4BC3-AAC5-F9A522442897,Kastrup søbad,Tårnby,Badevandsstation,Mangler,1990-05-07,Badevandsstation,32,730169.999999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18511,18495,6B0DA2AE-4E80-477C-8D00-BBE2EC7C385C,Dragør Nordstrand,Dragør,Badevandsstation,Nordlige Øresund,1990-05-07,Badevandsstation,32,731373.999999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
